# Comparison Auriga-MW [Fe/H] Profiles

In [13]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic

In [14]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse
from auriga.physics import Physics

In [15]:
figure_setup()
settings = Settings()

In [16]:
r_xy_range = (0, 16)
n_bins = np.diff(r_xy_range)

In [17]:
def read_snapshot_data(simulation: str, of: str, to: str, band: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.
    of : str
        The metal of which to calculate the metallicity.
    to : str
        The metal to which to calculate the metallicity.
    band : str
        The optical band for the luminosity calculation.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """

    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[4])
    s.add_extra_coordinates()
    s.add_metal_abundance(of=of, to=to)
    s.add_stellar_age()
    s.add_stellar_photometrics()
    s.add_luminosities(band=band)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df[f"[{of}/{to}]"] = s.metal_abundance[f"{of}/{to}"][
        is_real_star & is_main_obj]
    df["r_xy"] = s.rho[is_real_star & is_main_obj]
    df["StellarAge"] = s.stellar_age[is_real_star & is_main_obj]
    df["Luminosity_W"] = s.stellar_luminosities[is_real_star & is_main_obj]

    return df

In [18]:
def calc_mean_stat(x, y, weights = None):
    if weights is None:
        weights = np.ones(x.shape)

    num = binned_statistic(x=x, values=y * weights,
        statistic="sum", bins=n_bins, range=r_xy_range)[0]
    den = binned_statistic(x=x, values=weights,
        statistic="sum", bins=n_bins, range=r_xy_range)[0]
    mean = num / den

    std, bin_edges, _ = binned_statistic(
        x=x, values=y, statistic="std", bins=n_bins, range=r_xy_range)
    count, _, _ = binned_statistic(
        x=x, values=y, statistic="count", bins=n_bins, range=r_xy_range)
    err = std / np.sqrt(count)

    bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
    return bin_centers, mean, err

In [19]:
def read_experimental_data():
    df = pd.read_csv("../data/lian_2023/MW_Z_prof.txt", delimiter=' ')
    return df

In [20]:
def read_sample_data(galaxies: list, band: str) -> pd.DataFrame:
    df = pd.DataFrame()
    for galaxy in settings.galaxies:
        gdf = read_snapshot_data(simulation=f"au{galaxy}_or_l4_s127",
                                 of="Fe", to='H', band=band)
        bin_centers, mean, _ = calc_mean_stat(
            x=gdf["r_xy"][np.isfinite(gdf["[Fe/H]"])],
            y=gdf["[Fe/H]"][np.isfinite(gdf["[Fe/H]"])],
            weights=gdf["Luminosity_W"][np.isfinite(gdf["[Fe/H]"])])
        df[f"MeanFeH_Au{galaxy}"] = mean
    df["BinCenters_ckpc"] = bin_centers
    return df

In [21]:
df = read_sample_data([f"au{i}_or_l4_s127" for i in settings.galaxies],
                      band='r')
df_exp = read_experimental_data()

In [22]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))

ax.set_xlim(0, 16)
ax.set_ylim(-0.6, 0.6)
ax.set_xlabel(r"$r_{xy}$ [ckpc]")
ax.set_ylabel(r"$\left< \mathrm{[Fe/H]} \right>$")

ax.grid(True, ls='-', lw=0.25, c='silver')

# Weighted profile for all galaxies
for galaxy in settings.galaxies:
    ax.plot(df["BinCenters_ckpc"], df[f"MeanFeH_Au{galaxy}"],
            lw=0.7, zorder=10, c="gainsboro", ls='-')

ax.plot(df["BinCenters_ckpc"], df[f"MeanFeH_Au2"],
        lw=1.0, zorder=10, c="tab:green", ls='-',
        label=r"$\texttt{" + f"au2_or_l4_s127".upper() + "}$")
ax.plot(df["BinCenters_ckpc"], df[f"MeanFeH_Au6"],
        lw=1.0, zorder=10, c="tab:blue", ls='-',
        label=r"$\texttt{" + f"au6_or_l4_s127".upper() + "}$")

# Experimental data
ax.errorbar(df_exp["radius"], df_exp["total"], yerr=df_exp["total_err"],
            fmt="-o", capthick=0, color="tab:red", markersize=2.5,
            markeredgecolor="tab:red", zorder=10, label="Lian+2023",
            lw=0.7)

ax.legend(loc="lower left", framealpha=0.0, fontsize=8.0)

fig.savefig(f"../images/metallicity_profiles/mw_comparison/profiles.pdf")
plt.close(fig)

In [23]:
dfg = pd.read_csv("../data/grand_2017.csv")
mw_effective_radius_min = 3.4
mw_effective_radius_max = 6.7

In [24]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))

ax.set_xlim(0, 5)
ax.set_ylim(-0.6, 0.6)
ax.set_xlabel(r"$r_{xy}$ [$R_\mathrm{eff}$]")
ax.set_ylabel(r"$\left< \mathrm{[Fe/H]} \right>$")

ax.grid(True, ls='-', lw=0.25, c='silver')

# Weighted profile for all galaxies
for galaxy in settings.galaxies:
    effective_radius = float(dfg[
        "RadialScalelength_kpc"][dfg["Run"] == f"Au{galaxy}_L4"])
    ax.plot(df["BinCenters_ckpc"] / effective_radius,
            df[f"MeanFeH_Au{galaxy}"],
            lw=0.7, zorder=10, c="gainsboro", ls='-')

ax.plot(df["BinCenters_ckpc"] / float(dfg[
        "RadialScalelength_kpc"][dfg["Run"] == "Au2_L4"]),
        df[f"MeanFeH_Au2"],
        lw=1.0, zorder=10, c="tab:green", ls='-',
        label=r"$\texttt{" + f"au2_or_l4_s127".upper() + "}$")
ax.plot(df["BinCenters_ckpc"] / float(dfg[
        "RadialScalelength_kpc"][dfg["Run"] == "Au6_L4"]),
        df[f"MeanFeH_Au6"],
        lw=1.0, zorder=10, c="tab:blue", ls='-',
        label=r"$\texttt{" + f"au6_or_l4_s127".upper() + "}$")

# Experimental data
ax.errorbar(df_exp["radius"] / mw_effective_radius_min, df_exp["total"],
            yerr=df_exp["total_err"],
            fmt="-o", capthick=0, color="tab:red", markersize=2.5,
            markeredgecolor="tab:red", zorder=10, label="Lian+2023",
            lw=0.7)
ax.errorbar(df_exp["radius"] / mw_effective_radius_max, df_exp["total"],
            yerr=df_exp["total_err"],
            fmt="-o", capthick=0, color="tab:red", markersize=2.5,
            markeredgecolor="tab:red", zorder=10, lw=0.7)

ax.legend(loc="upper right", framealpha=0.0, fontsize=8.0)

fig.savefig(
    f"../images/metallicity_profiles/mw_comparison/profiles_eff_radius.pdf")
plt.close(fig)